First we read our data from our csv file. We choose the `subject`,
`sessionIndex`, and `rep` columns as our index columns.

| subject | sessionIndex | rep | ... |
|---------|:------------:|:---:|-----|
| s002    | 1            | 1   | ... |
| s002    | 1            | 2   | ... |
| ...     | ...          | ... | ... |
| s002    | 2            | 1   | ... |
| ...     | ...          | ... | ... |
| s003    | 1            | 1   | ... |

In [2]:
import gudhi as gd
import pandas as pd

passwords_data_frame = pd.read_csv('data/DSL-StrongPasswordData.csv',
                                   # declare type of first column
                                   dtype = {'subject' : 'string'},
                                   index_col = ['subject', 'sessionIndex', 'rep'])

We group the first four people (subjects column in csv file). Subjects `s006`,
..., are missing, so we use why I used.

In [5]:
people = [passwords_data_frame.loc[f's{n:03}'] for n in range(2,6)]

In [8]:
persistence_diagrams = []

for person in people:
    simplicial_complex = gd.RipsComplex(points = person.values.tolist(),
                                        max_edge_length = 0.2)
    simplex_tree = simplicial_complex.create_simplex_tree(max_dimension = 4)
    persistence_diagrams.append(simplex_tree.persistence())

In [ ]:
#THIS CODE NO LONGER WORKS PAST THIS POINT. See the first file that was posted to discord 
# for working code that flows through the whole notebook.
#
# BarCodes_0 = simplextree_0.persistence()
# BarCodes_1 = simplextree_1.persistence()
# print(len(BarCodes_0))
# print(len(BarCodes_1))
#

In [ ]:
# for i in range(20):
#     print(BarCodes_Rips0)
#

In [ ]:
# gd.plot_persistence_diagram(BarCodes_0);
# gd.plot_persistence_diagram(BarCodes_1);
#

In [ ]:
# for i in range(0,3):
#
#     I0 = simplextree_0.persistence_intervals_in_dimension(i)
#     I1 = simplextree_1.persistence_intervals_in_dimension(i)
#
#     print(len(I0))
#     print(len(I1))
#     print(gd.bottleneck_distance(I0,I1))
#

In [ ]:
# for i in range(57):
#     for j in range(i):
#         distance[i,j] = gd.bottleneck_distance(zerobarcodes[i], zerobarcodes[j])
#         distance[i,j] = gd.bottleneck_distance(zerobarcodes[i], zerobarcodes[j])
# distance=distance + distance.transpose()
# print(distance)
#